### 1. General info of dataset GSE137804

This is the Jupyter Notebook for dataset GSE137804. Its dataset includes a xls file for each sample. As seen below, in the txt file, each row is a gene and each column is a cell.

Thus, we need to transform this txt file and generate the AnnData object for each sample. In total, there are 22 samples.

In [237]:
# Environment setup
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as anndata
import scipy

In [258]:
# inspect one tumor dataset
tumor_path = '/scratch/user/s4543064/xiaohan-john-project/data/GSE137804/GSM4088774_T10_gene_cell_exprs_table.xls.gz'
tumor_input = pd.read_csv(tumor_path, sep='\t', index_col=1) # the second column contains gene names and is the index

print(tumor_input.head()) 
print(tumor_input.shape) 

                  Gene_ID  AAACCTGAGCTCAACT.1  AAACCTGAGGAGCGTT.1  \
Symbol                                                              
TSPAN6    ENSG00000000003                   1                   0   
DPM1      ENSG00000000419                   0                   1   
SCYL3     ENSG00000000457                   0                   0   
C1orf112  ENSG00000000460                   0                   0   
FGR       ENSG00000000938                   0                   0   

          AAACCTGCACCAGATT.1  AAACCTGGTCAGAATA.1  AAACCTGTCTTTAGTC.1  \
Symbol                                                                 
TSPAN6                     0                   0                   0   
DPM1                       0                   1                   0   
SCYL3                      0                   0                   0   
C1orf112                   0                   0                   0   
FGR                        0                   0                   0   

          A

In [259]:
# inspect the tumor annotation file
tumor_annotation_path = '/scratch/user/s4543064/xiaohan-john-project/data/GSE137804/GSE137804_tumor_dataset_annotation.csv.gz'
tumor_annotation = pd.read_csv(tumor_annotation_path, sep=',', index_col=0) 

tumor_annotation

,sample,celltype
cellname,,
T10_AAGACCTAGTACTTGC.1,T10,tumor
T10_TTTACTGCATGTAGTC.1,T10,tumor
T10_AGAGCGAAGTTGAGTA.1,T10,tumor
T10_CACCTTGAGTGTGGCA.1,T10,tumor
T10_GAAATGAGTGGTACAG.1,T10,tumor
...,...,...
T230_TGTAACGTCCTAAGTG-1,T230,Tcell
T230_TTCATGTGTGATAGTA-1,T230,Tcell
T230_TTCGGTCAGTCGAAAT-1,T230,Tcell


The index 'cellname' has a prefix, which needs to be removed during processing

In [262]:
# inspect the embryo dataset
embryo_path = '/scratch/user/s4543064/xiaohan-john-project/data/GSE137804/GSM4654667_F366_gene_cell_exprs_table.xls.gz'
embryo_input = pd.read_csv(embryo_path, sep='\t', index_col=0) # the second column contains gene names and is the index

print(embryo_input.head()) 

             AAACCCAAGACTAAGT.1  AAACCCAAGAGGCGGA.1  AAACCCAAGGATACCG.1  \
MIR1302-2HG                   0                   0                   0   
FAM138A                       0                   0                   0   
OR4F5                         0                   0                   0   
AL627309.1                    0                   0                   0   
AL627309.3                    0                   0                   0   

             AAACCCAAGGTTGGAC.1  AAACCCAAGTCAGGGT.1  AAACCCAAGTCTACCA.1  \
MIR1302-2HG                   0                   0                   0   
FAM138A                       0                   0                   0   
OR4F5                         0                   0                   0   
AL627309.1                    0                   0                   0   
AL627309.3                    0                   0                   0   

             AAACCCAAGTCTCCTC.1  AAACCCACAAGCACCC.1  AAACCCACAAGTGCAG.1  \
MIR1302-2HG            

In [263]:
# inspect the embryo annotation file
embryo_annotation_path = '/scratch/user/s4543064/xiaohan-john-project/data/GSE137804/GSE137804_Embryo_annotation.csv.gz'
embryo_annotation = pd.read_csv(embryo_annotation_path, sep=',', index_col=0) 

embryo_annotation

,sample,annotation
cell_id,,
AAACCCAAGATTGAGT.1,F379,Mesenchyme
AAACCCACAACCGTGC.1,F379,Motor neuron
AAACCCACAAGAAATC.1,F379,Mesenchyme
AAACCCACACGGTGCT.1,F379,Cadiomycytes
AAACCCACATGAATCC.1,F379,Neural crest
...,...,...
TTTGTTGTCCATGATG.1,F366,Mesenchyme
TTTGTTGTCCCATTTA.1,F366,Mesenchyme
TTTGTTGTCCGCATAA.1,F366,Mesenchyme


The count matrix file for embryos is different from the one for tumor samples, so need to process separately

In [265]:
# inspect the adrenal dataset
adrenal_path = '/scratch/user/s4543064/xiaohan-john-project/data/GSE137804/GSM4088785_F2_gene_cell_exprs_table.xls.gz'
adrenal_input = pd.read_csv(adrenal_path, sep='\t', index_col=1) # the second column contains gene names and is the index

print(adrenal_input.head()) 

                  Gene_ID  AAACCTGAGACCTTTG.1  AAACCTGAGCGTTGCC.1  \
Symbol                                                              
TSPAN6    ENSG00000000003                   1                   0   
TNMD      ENSG00000000005                   0                   0   
DPM1      ENSG00000000419                   0                   0   
SCYL3     ENSG00000000457                   0                   0   
C1orf112  ENSG00000000460                   0                   0   

          AAACCTGAGGAATTAC.1  AAACCTGAGGTGCAAC.1  AAACCTGAGTACTTGC.1  \
Symbol                                                                 
TSPAN6                     1                   0                   1   
TNMD                       0                   0                   0   
DPM1                       1                   0                   1   
SCYL3                      0                   0                   0   
C1orf112                   0                   0                   0   

          A

In [266]:
# inspect the adrenal annotation file
adrenal_annotation_path = '/scratch/user/s4543064/xiaohan-john-project/data/GSE137804/GSE137804_Adrenal_gland_annotation.csv.gz'
adrenal_annotation = pd.read_csv(adrenal_annotation_path, sep=',', index_col=0) 

adrenal_annotation

,sample,annotation
cell_id,,
AAATGCCAGCTGTCTA.1_1,F2,Steroidogenic cells
AAATGCCTCACAATGC.1_1,F2,Steroidogenic cells
AACACGTGTCATGCCG.1_1,F2,Steroidogenic cells
AACCGCGAGATCTGAA.1_1,F2,Steroidogenic cells
AACGTTGTCGCTTAGA.1_1,F2,Steroidogenic cells
...,...,...
TTGACTTGTGTTCGAT.1_4,F107,SCPs
TTGGCAAAGAGGTTAT.1_4,F107,SCPs
TTGTAGGAGGCGTACA.1_4,F107,SCPs


In [267]:
if 'AAATGCCAGCTGTCTA.1_1' in adrenal_input.columns:
    print('The cell name is matched')
else:
    print('The cell is MISMATCHED')

The cell is MISMATCHED


In [268]:
if 'AAATGCCAGCTGTCTA.1' in adrenal_input.columns:
    print('The cell name is matched')
else:
    print('The cell is MISMATCHED')

The cell name is matched


<span style="color:red">**PROBLEM:**</span> the cell names between the count matrix file and the annotation file are different

### 2. Overall AnnData object of the dataset

<span style="color:red">**IMPORTANT:**</span> transpose the DataFrame.values to match the AnnData.X

1. `DataFrame.columns`: cell barcodes, which go into `.obs`
2. `DataFrame.index`: gene names, `.var`
3. `DataFrame.values`: the transpose of the expression matrix, `.X`

In [273]:
# Load the metadata

tissue_dict = {
    'GSM4088774': 'adrenal_gland', 
    'GSM4088775': 'adrenal_gland',
    'GSM4088776': 'adrenal_gland',
    'GSM4088777': 'adrenal_gland',
    'GSM4088778': 'adrenal_gland',
    'GSM4088779': 'adrenal_gland',
    'GSM4088780': 'adrenal_gland',
    'GSM4088781': 'adrenal_gland',
    'GSM4088782': 'adrenal_gland',
    'GSM4088783': 'adrenal_gland',
    'GSM4088785': 'adrenal_gland',
    'GSM4088786': 'adrenal_gland',
    'GSM4088787': 'adrenal_gland',
    'GSM4088788': 'adrenal_gland',
    'GSM4654667': 'embryo', 
    'GSM4654668': 'embryo',
    'GSM4654669': 'adrenal_gland',
    'GSM4654670': 'adrenal_gland',
    'GSM4654671': 'adrenal_gland',
    'GSM4654672': 'adrenal_gland',
    'GSM4654673': 'adrenal_gland',
    'GSM4654674': 'adrenal_gland',
}

sex_age_cancer_dict = {
    'T10': ['male', 3, 'ganglioneuroblastoma'],
    'T19': ['female', 2, 'ganglioneuroblastoma'],
    'T27': ['male', 3, 'neuroblastoma'],
    'T34': ['female', 3, 'neuroblastoma'],
    'T40': ['male', 0.17, 'neuroblastoma'],
    'T44': ['female', 0.17, 'neuroblastoma'],
    'T69': ['male', 2, 'neuroblastoma'],
    'T71': ['male', 0.01, 'neuroblastoma'],
    'T75': ['female', 0.75, 'neuroblastoma'],
    'T92': ['female', 3, 'neuroblastoma'],
    'T162': ['male', 2, 'neuroblastoma'],
    'T174': ['male', 0.06, 'neuroblastoma'],
    'T188': ['male', 0.25, 'neuroblastoma'],
    'T200': ['female', 2, 'neuroblastoma'],
    'T214': ['female', 0.25, 'neuroblastoma'],
    'T230': ['male', 4, 'neuroblastoma'],
}

In [272]:
from pathlib import Path

# Specify directory paths
data_directory = Path('/scratch/user/s4543064/xiaohan-john-project/data/GSE137804')

# Loop through all files in the directory
for file in data_directory.iterdir():
    if 'annotation' not in file.stem:
        gsm, patient_id = file.stem.split('_')[:2]
        
        if gsm in ['GSM4654667', 'GSM4654668']: # it is embryo sample
            input = pd.read_csv(file, sep='\t', index_col=0)

            cancer_type = 'neuroblastoma'
            tissue = 'unknown'
        else:
            input = pd.read_csv(file, sep='\t', index_col=1)
            gene_id_df = input.loc[:, ['Gene_ID']]
            input.drop(columns=['Gene_ID'], inplace=True)

            cancer_type = 
            tissue =
            age = 


        matrix = scipy.sparse.csr_matrix(input.values.T)
        obs_name = pd.DataFrame(index=input.columns)
        var_name = gene_id_df
        var_name.rename_axis('gene_symbols', inplace=True)

        sample = anndata.AnnData(X=matrix, obs=obs_name, var=var_name)

        # Create an observation metric info to store related features
        obs_metrics = pd.DataFrame(index=sample.obs_names) ## Get the identifiers
        obs_metrics['cancer_type'] = 'Ganglioneuroma' 
        obs_metrics['dataset'] = 'GSE137804'
        obs_metrics['tissue'] = 'adrenal'
        obs_metrics['sample_barcode'] = file.stem
        obs_metrics['uni_barcode'] = obs_metrics['dataset'] + '_' + obs_metrics.index.astype(str)

        sample.obs = obs_metrics
        sample.obs.set_index("uni_barcode", drop=False, inplace=True)
        print(sample)

        # save the anndata object
        sample.write_h5ad(f'/scratch/user/uqjsaxo1/xiaohan-john-project/write/GSE192906/{file.stem}_uni.h5ad', compression="gzip")

GSM4088780 T69


In [254]:
obs_name

""
AAACCTGAGACGCAAC.1
AAACCTGAGACGCACA.1
AAACCTGAGACTACAA.1
AAACCTGAGAGAGCTC.1
AAACCTGAGCAGACTG.1
...
TTTGTCATCACATACG.1
TTTGTCATCCAAACTG.1
TTTGTCATCCTGCAGG.1
TTTGTCATCGTTTATC.1


In [250]:
input

,AAACCTGAGACGCAAC.1,AAACCTGAGACGCACA.1,AAACCTGAGACTACAA.1,AAACCTGAGAGAGCTC.1,AAACCTGAGCAGACTG.1,AAACCTGAGCCCGAAA.1,AAACCTGAGTACGCGA.1,AAACCTGAGTATTGGA.1,AAACCTGCACAGGAGT.1,AAACCTGCACATCCGG.1,...,TTTGTCAGTAGAGTGC.1,TTTGTCAGTCTCGTTC.1,TTTGTCAGTGCATCTA.1,TTTGTCAGTGCTAGCC.1,TTTGTCATCAAGCCTA.1,TTTGTCATCACATACG.1,TTTGTCATCCAAACTG.1,TTTGTCATCCTGCAGG.1,TTTGTCATCGTTTATC.1,TTTGTCATCTTACCTA.1
Symbol,,,,,,,,,,,,,,,,,,,,,
TSPAN6,0,0,0,0,0,0,0,0,0,0,...,4,0,0,0,0,0,1,1,0,1
DPM1,0,0,0,0,0,1,0,0,0,0,...,0,0,1,1,1,0,2,1,2,1
SCYL3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C1orf112,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
FGR,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RP1-34B20.21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RP11-511P7.5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RP11-736N17.11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
sample.obs

,cancer_type,dataset,tissue,sample_barcode,uni_barcode
uni_barcode,,,,,
GSE192906_A1_1_0314_AACGAGGT,Ganglioneuroblastoma,GSE192906,Neuronal,GSM5768750_GNB3_UMI_COUNTS_RAW,GSE192906_A1_1_0314_AACGAGGT
GSE192906_A1_1_0314_AAGCACAT,Ganglioneuroblastoma,GSE192906,Neuronal,GSM5768750_GNB3_UMI_COUNTS_RAW,GSE192906_A1_1_0314_AAGCACAT
GSE192906_A1_1_0314_ACACCGTG,Ganglioneuroblastoma,GSE192906,Neuronal,GSM5768750_GNB3_UMI_COUNTS_RAW,GSE192906_A1_1_0314_ACACCGTG
GSE192906_A1_1_0314_ACCTCAGC,Ganglioneuroblastoma,GSE192906,Neuronal,GSM5768750_GNB3_UMI_COUNTS_RAW,GSE192906_A1_1_0314_ACCTCAGC
GSE192906_A1_1_0314_ACTGTTTG,Ganglioneuroblastoma,GSE192906,Neuronal,GSM5768750_GNB3_UMI_COUNTS_RAW,GSE192906_A1_1_0314_ACTGTTTG
...,...,...,...,...,...
GSE192906_K2_2_0314_TCTCACAC,Ganglioneuroblastoma,GSE192906,Neuronal,GSM5768750_GNB3_UMI_COUNTS_RAW,GSE192906_K2_2_0314_TCTCACAC
GSE192906_K2_2_0314_TGGAGCTC,Ganglioneuroblastoma,GSE192906,Neuronal,GSM5768750_GNB3_UMI_COUNTS_RAW,GSE192906_K2_2_0314_TGGAGCTC
GSE192906_K2_2_0314_TGTACCAA,Ganglioneuroblastoma,GSE192906,Neuronal,GSM5768750_GNB3_UMI_COUNTS_RAW,GSE192906_K2_2_0314_TGTACCAA


### 3. Confirmation of created AnnData object

In [13]:
output = '/scratch/user/uqjsaxo1/xiaohan-john-project/write/GSE192906/GSM5768743_NB1_UMI_COUNTS_RAW_uni.h5ad'
sample = anndata.read_h5ad(output)
print(sample)

AnnData object with n_obs × n_vars = 960 × 33514
    obs: 'cancer_type', 'dataset', 'tissue', 'sample_barcode', 'uni_barcode'
    var: 'gene_symbols'


### 4. Convert AnnData objects to SingleCellExperiment objects

In [14]:
from pathlib import Path

import anndata2ri
import rpy2.robjects as robjects
from rpy2.robjects import r
from rpy2.robjects.conversion import localconverter

# Specify directory paths
write_directory = Path('/scratch/user/s4543064/xiaohan-john-project/write/GSE102130')

# Loop through all files in the directory
for file in write_directory.iterdir():
    sample_name = file.stem
    if "_uni.h5ad" in file.name:
        sample_anndata = anndata.read_h5ad(file)
        sample_sce_file = sample_name + ".rds"

        with localconverter(anndata2ri.converter):
            sample_sce = anndata2ri.py2rpy(sample_anndata)
        print(sample_sce)
        
        # Save the sce object in .rds file
        robjects.globalenv["sample_sce"] = sample_sce
        sample_sce_path = write_directory / sample_sce_file
        robjects.r("saveRDS(sample_sce, file='{}')".format(sample_sce_path))